In [ ]:
import uproot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import rcParams
import seaborn as sns
import scienceplots
sns.set(font_scale=1.0)
sns.set_style('white')
#plt.style.use('fivethirtyeight') # fivethirtyeight is name of styl
#plt.style.use(['science','no-latex'])
#comparing the result in this presentation slide 4
#https://docs.google.com/presentation/d/14KgpLBpJwNQH5tjEczl9tPkInSiKkJtjgqBPWSyoXok/edit#slide=id.g1b79411b324_0_80

In [ ]:
rcParams.update({'figure.autolayout':True})
rcParams.update({'figure.figsize':[12,8]})
plt.rcParams['legend.fontsize'] = 12

# HELPER FUNCTION

In [ ]:
def my_plotter(ax,ene,weight,label_value,yscale,xlabel='Energy [keV]',ylabel='Counts',iso="Th228"):
    """
    A helper function to make a graph

    Parameters
    ----------
    ax : Axes
        The axes to draw to

    data1 : array
       The x data

    data2 : array
       The y data

    param_dict : dict
       Dictionary of keyword arguments to pass to ax.plot

    Returns
    -------
    out : list
        list of artists added
    """
    out=ax.hist(ene, bins=200, histtype=u'step', weights=weight, density=True, label=label_value)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yscale(yscale)
    ax.set_title(f"Energy PDF for SS events  for {iso}")
    ax.legend()
    
    return out

In [ ]:
#input the root file and ouput the filtered dataframe
#apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'

def get_filtered(file,apply_filter,use_filter="True"):
    f=uproot.open(file+':tree')
    df=f.arrays(f.keys(),library='pd')
    if use_filter:return df.query(apply_filter).reset_index(drop=True)
    return df.query('energy<3000').reset_index(drop=True)
    
    


In [ ]:
# returns the plot for the isotopes in the given dataframe 
def plot_isotopes(df,df_label="PX",scale_kind="log"):
    #isotopes=df.isotope.unique()
    ra224_df=df.query('isotope==1')
    pb212_df=df.query('isotope==2')
    bi212_df=df.query('isotope==3')
    tl208_df=df.query('isotope==4')
    
    ra224_energy,ra224_weight=ra224_df.energy,ra224_df.weight
    pb212_energy,pb212_weight=pb212_df.energy,pb212_df.weight
    bi212_energy,bi212_weight=bi212_df.energy,bi212_df.weight
    tl208_energy,tl208_weight=tl208_df.energy,tl208_df.weight
    
    fig, ax = plt.subplots(1, 1,figsize=(6,4))
    my_plotter(ax,ra224_energy,ra224_weight,df_label+" Ra224",scale_kind)
    my_plotter(ax,pb212_energy,pb212_weight,df_label+" Pb212",scale_kind)
    my_plotter(ax,bi212_energy,bi212_weight,df_label+" Bi212",scale_kind)
    my_plotter(ax,tl208_energy,tl208_weight,df_label+" Tl208",scale_kind)
    plt.show()
    

# ROOT DATA FILES

In [ ]:
#give the name of rootfile
#th228_px='/home/thakur/slac_data/PXE05M/PXE05M_Th228_pos648.5_0_-1022.6.root'
th228_all='/home/thakur/slac_data/th228/th228_Th228_all.root'
#th228_all='/home/thakur/slac_data/v3th228/v3th228_Th228_all.root'
# th228_nx='/home/thakur/slac_data/NX1M/NX1M_Th228_pos-648.5_0_-1022.6.root'
# th228_py='/home/thakur/slac_data/PY1M/PY1M_Th228_pos0_648.5_-1022.6.root'
# th228_ny='/home/thakur/slac_data/NY1M/NY1M_Th228_pos0_-648.5_-1022.6.root'
# th228_pz='/home/thakur/slac_data/PZ1M/PZ1M_Th228_pos0_0_-299.1245.root'
# th228_nz='/home/thakur/slac_data/NZ1M/NZ1M_Th228_pos0_0_-1746.0755.root'

# WITHOUT FILTER

In [ ]:
non_filtered_th228=get_filtered(th228_all,dec102020_filter,use_filter=False)
# non_filtered_th228_nx=get_filtered(th228_nx,dec102020_filter,use_filter=False)
# non_filtered_th228_py=get_filtered(th228_py,dec102020_filter,use_filter=False)
# non_filtered_th228_ny=get_filtered(th228_ny,dec102020_filter,use_filter=False)
# non_filtered_th228_pz=get_filtered(th228_pz,dec102020_filter,use_filter=False)
# non_filtered_th228_nz=get_filtered(th228_nz,dec102020_filter,use_filter=False)
non_filtered_th228

In [ ]:
non_filtered_th228.head()

In [ ]:
non_filtered_th228.columns

# FILTER

In [ ]:
#input the root file and ouput the filtered dataframe
#apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'


dec102020_filter='(standoff>100) & passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < (1.077 * m_nQ + 313)) & (m_nOPCal > (0.597 * m_nQ - 216)) & ~NESTBugFound & ~NearAnodeBugFound & m_DNNvalue>0.85'
jason_filter='''
(standoff>100) &\
m_DNNvalue>0.85 &\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216))&\
(abs(energy-2614)<10)
'''
# & ~NESTBugFound & ~NearAnodeBugFound' & (abs(energy-2614)<10)'''


In [ ]:
 #this is the filter used by jason
check=(
(non_filtered_th228['standoff']>100)\
& (non_filtered_th228['m_DNNvalue']>0.850)\
& (non_filtered_th228['n_x_ch_abovenoise']>0)\
& (non_filtered_th228['n_y_ch_abovenoise']>0)\
& (non_filtered_th228['passed_xy_thresh']==1)\
& (non_filtered_th228['passed_z_thresh']==1)\
& (non_filtered_th228['m_nOPCal']<(1.077*non_filtered_th228['m_nQ']+313))\
& (non_filtered_th228['m_nOPCal']>(0.597*non_filtered_th228['m_nQ']-216))\
& (abs(non_filtered_th228['energy']-2614)<10)
  )

In [ ]:
non_filtered_th228_filtered=non_filtered_th228[check]
non_filtered_th228_filtered

In [ ]:
#total events in g4 tree without application of any cuts
events_in_g4tree=non_filtered_th228.shape[0]
events_in_g4tree

# $\alpha$, $\gamma$ and $A$
$ \alpha = \frac{\text{events in g4tree}}{\text{total number of simulations}} $
- Need to track the empty root files for some lower rate isotopes


$ R(A)= \gamma A (1+\alpha A t ) e^{-\alpha A t}$
- $\alpha $ ratio of good events
- $\gamma$ SS events
- t electron life time ($10e3$)

$ A=\frac{1+\sqrt{5}}{2\times \alpha \times t}=\frac{1.618}{\alpha \times t}$

In [ ]:
t=10e3
#good_files=174;events_per_file=20_000
total_simulations=1000_000
#total_simulations=good_files*events_per_file
print(f"Events in g4tree : {events_in_g4tree:.3f}")
print(f"Total simulations: {total_simulations:.3f}")

alpha=events_in_g4tree/total_simulations
A=1.618/(alpha*t)

print(f"Alpha            : {alpha:.3f}")
print(f"A                : {A:.3f}")

In [ ]:
#unique isotopes
list_iso=non_filtered_th228.isotope.unique()
list_iso


In [ ]:
#number of individual isotopes in the g4 tree without any application of cuts
non_iso=non_filtered_th228.isotope.value_counts().to_list()[::-1]
non_iso

In [ ]:
isotopes=['Th228','Ra224','Pb212','Bi212','Tl208']
isotopes

In [ ]:
#isotope coutns before the cut
raw_counts_df=pd.DataFrame(non_filtered_th228.isotope.value_counts().reset_index().values,columns=['isonum','raw_counts'])\
.sort_values(by=['isonum'])\
.reset_index(drop=True)

raw_counts_df["isotopes"]=raw_counts_df.isonum.apply(lambda x:isotopes[x-1])
raw_counts_df=raw_counts_df[['isonum','isotopes','raw_counts']]
#raw_counts.index=raw_counts.isotopes
#print(raw_counts.to_string())
raw_counts_df

In [ ]:
# this information is taken from jaons's slide 5 (https://docs.google.com/presentation/d/14KgpLBpJwNQH5tjEczl9tPkInSiKkJtjgqBPWSyoXok/edit#slide=id.p)
jason_deposited=[0,27,238,7352,192258]

In [ ]:
df_deposited=pd.DataFrame({"my_result":non_iso,"jason's_result":jason_deposited},index=isotopes)

In [ ]:
df_deposited

# Deposited number of isotopes agree

In [ ]:
skipEThreshold=100
plt.figure(figsize=(6,4))
ax=df_deposited.plot.bar()
for c in ax.containers:
    # set the bar label
    ax.bar_label(c, fmt='%.0f', label_type='edge')
plt.xlabel('ISOTOPES')
plt.title(f'Isotope counts in g4tree for 1M Th228 at all positions for skipEThreshold: {skipEThreshold} keV')
plt.ylabel('COUNTS')
plt.yscale('log')
#plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
plt.show()

In [ ]:
iso_br=[1,1,1,1,0.3594] #branching ratios of Th228 to other isotopes


In [ ]:
iso_product=[a*b for a,b in zip(non_iso,iso_br)]
sum(iso_product)

In [ ]:
#number of isotope counts in the cut result
#after check cut above
df_check_cut=non_filtered_th228[check]
df_check_cut

In [ ]:
#same result with query function
df_check_query=non_filtered_th228.query(jason_filter)
df_check_query

In [ ]:
#good (Tl208 2.6 MeV) isotopes counts
df_check_cut.isotope.value_counts()

In [ ]:
#good counts dataframe
#isotope coutns before the cut
good_counts_df=pd.DataFrame(df_check_cut.isotope.value_counts().reset_index().values,columns=['isonum','good_counts'])\
.sort_values(by=['isonum'])\
.reset_index(drop=True)

good_counts_df["isotopes"]=good_counts_df.isonum.apply(lambda x:isotopes[x-1])
good_counts_df=good_counts_df[['isonum','isotopes','good_counts']]
#raw_counts.index=raw_counts.isotopes
#print(raw_counts.to_string())
good_counts_df

In [ ]:
#combining raw and good counts daa frames
combined_counts_df=raw_counts_df.merge(good_counts_df,how='left')
combined_counts_df=combined_counts_df[['isotopes','raw_counts','good_counts']]
combined_counts_df

In [ ]:
sim_num=1000_000
combined_counts_df.insert(loc=1,column="sim_events",value=5*[sim_num])
combined_counts_df

In [ ]:
#adding alpha columns
#alpha=events_in_g4tree/total_simulations
combined_counts_df['alpha']=combined_counts_df['raw_counts']/combined_counts_df['sim_events']
combined_counts_df

In [ ]:
#adding gamma column
#gamma is the good_counts or ss events to the total simulations
combined_counts_df['gamma']=combined_counts_df['good_counts']/combined_counts_df['sim_events']
combined_counts_df

In [ ]:
#adding branching ratio
combined_counts_df['branching_ratio']=iso_br
combined_counts_df.fillna(0,inplace=True)          #replacing NaN with 0
combined_counts_df 

In [ ]:
#zip and sum and adding a new row
raw_br=sum([a*b for a,b in zip(combined_counts_df.raw_counts,combined_counts_df.branching_ratio)])
good_br=sum([a*b for a,b in zip(combined_counts_df.good_counts,combined_counts_df.branching_ratio)])
alpha_br=sum([a*b for a,b in zip(combined_counts_df.alpha,combined_counts_df.branching_ratio)])
gamma_br=sum([a*b for a,b in zip(combined_counts_df.gamma,combined_counts_df.branching_ratio)])

#adding a row
combined_counts_df.loc[len(combined_counts_df.index)]=['Th228_Chain',1000000,raw_br,good_br,alpha_br,gamma_br,1]

In [ ]:
#final table
combined_counts_df.rename(columns={"isotopes":"sources","raw_counts":"deposited"},inplace=True)
combined_counts_df

In [ ]:
print(f"""
=================================== SUMMARY TABLE =======================================

{combined_counts_df.to_string()}

=========================================================================================
""")

# Rate Estimation

In [ ]:
#for Th228_Chain
# alpha=alpha_br;gamma=gamma_br;t=10e3;
# alpha=0.0767
# gamma=3.3e-5
gamma=1.99e-5
alpha=4.63e-1
t=1e-3
A=1.618/(alpha*t)

print(f"""
alpha: {alpha:e}
gamma: {gamma:e}
t    : {t:e}
A    : {A:e}
       """)

In [ ]:
# import numpy as np
# (1+numpy.sqrt(5))/(2*alpha*t)

$ R(A)= \gamma A (1+\alpha A t ) e^{-\alpha A t}$
- $\alpha $ ratio of good events
- $\gamma$ SS events
- t electron life time ($10e3$)

In [ ]:
# #R(A) calculaitons
# import math
# R_A=gamma*A*(1+alpha*A*t)*math.exp(-alpha*A*t)
# print(f"R_A: {R_A}")

In [ ]:
# #R(A) calculaitons
# import math
# def get_R_A(A):
#     R=gamma*A*(1+alpha*A*t)*math.exp(-alpha*A*t)
#     if R<R_A:print("less!")
#     else:print(f"R_A: {R}")

In [ ]:
# x=np.arange(500)
# for i in x:
#     get_R_A(i)
#
#t=10e3/1000
t=1e-3 # the electron lifetime in simulation is 10^4 us
#t1=1e-2
alpha=0.0767
gamma=3.3e-5
# alpha=0.069
# gamma=1.29e-7
#x=np.arange(50)  #x in kBq
A_min=1.618/(alpha*t)

A_optimal=(1+np.sqrt(5))/(2*alpha*t)     


print(f"Optimal source strength : {A_optimal/1000:0.2f} kBq\n")
print(f"A_min                   : {A_min/1000:0.2f} kBq\n")

x_range_max=50
x=np.linspace(1,x_range_max,5000)
#y=[gamma*i*(1+alpha*i*t)*np.exp(-alpha*i*t) for i in x*1000]
y=np.array([gamma*i*np.exp(-alpha*i*t) for i in x*1000])#x is multiplied by 1000 to change to Bq
#y_max=np.max(y)
arg_y_max=np.argmax(y)
arg_y_min=np.argmin(y)

y_max=y[arg_y_max]
y_min=y[arg_y_min]

# print(f"A_min                   : {A_min:0.2f}")
# print(f"A_max_index             : {arg_max:0.2f}")
# print(f"A_max                   : {y_max:0.2f}")
# print(f"Min_rate                : {y[arg_y_min]:0.2f} Hz ({y[arg_max]*60:0.2f} Evetns/min)")
#print(f"Activity for Max Rate   : {x[arg_max]:0.2f} kBq\n")
print(f"Max_rate                : {y_max:0.2f} Hz ({y_max*60:0.1f} Evetns/min)")
print(f"Activity for Max Rate   : {x[arg_y_max]:0.2f} kBq\n")

#plt.axvline(x=A,color='r',linestyle='--')
plt.xlabel('Activity [kBq]')
plt.ylabel('Rate [Hz]')
plt.grid()
plt.plot(x,y)
plt.show()

# [plot on the slide 1/16](https://nexowiki.llnl.gov/images/d/dd/171212-extcal-calib.pdf)

In [ ]:

#t=10e3/1000
t=1e-3 # the electron lifetime in simulation is 10^4 us
alpha=4.63e-1
beta=2.10e-2
gamma=1.99e-5


# A_optimal=(1+np.sqrt(5))/(2*alpha*t)


# print(f"Optimal source strength : {A_optimal:0.2f} Bq")

x_range_max=10_000
x=np.linspace(1,x_range_max,5000)                    #defination of x

y1=np.array([gamma*i*np.exp(-beta*i*t) for i in x])

y2=np.array([gamma*i*np.exp(-alpha*i*t) for i in x])      #x is multiplied by 1000 to change to Bq
y3=np.array([gamma*i*(1+alpha*i*t)*np.exp(-alpha*i*t) for i in x])
#y_max=np.max(y)
arg_y_max=np.argmax(y)
arg_y_min=np.argmin(y)

y_max=y[arg_y_max]
y_min=y[arg_y_min]

# # print(f"A_min                   : {A_min:0.2f}")
# # print(f"A_max_index             : {arg_max:0.2f}")
# # print(f"A_max                   : {y_max:0.2f}")
# # print(f"Min_rate                : {y[arg_y_min]:0.2f} Hz ({y[arg_max]*60:0.2f} Evetns/min)")
# #print(f"Activity for Max Rate   : {x[arg_max]:0.2f} kBq\n")
# print(f"Max_rate                : {y_max:0.5f} Hz ({y_max*60:0.3f} Evetns/min)")
# print(f"Activity for Max Rate   : {x[arg_y_max]:0.3f} kBq\n")

fig,axs=plt.subplots(2)

legends=[fr'1 deep + $\geq$ 1 deep','1 deep + $\geq$ 1 shallow','1 deep + $\geq$ 2 shallow']
axs[0].plot(x,y1,'b',label=legends[0])
axs[0].plot(x,y2,'r',label=legends[1])
axs[0].plot(x,y3,'pink',label=legends[2])
axs[0].set_xlabel('Activity [kBq]')
axs[0].set_ylabel('Rate [Hz]')
axs[0].grid()
axs[0].legend()

axs[1].plot(x,1/y1*1/60,'b',label=legends[0])
axs[1].plot(x,1/y2*1/60,'r',label=legends[1])
axs[1].plot(x,1/y3*1/60,'pink',label=legends[2])
axs[1].set_xlabel('Activity [kBq]')
axs[1].set_ylabel('Time [s]')
axs[1].set_yscale('log')
axs[1].grid()
axs[1].legend()
#plt.plot(x,1/y3)
plt.show()

In [ ]:
y1

In [ ]:
1/y1

In [ ]:
1/y1*1/60

In [ ]:
1/(1.98995821e-05*60)

# Deposited Events plot without B.R. considerations

In [ ]:
iso=['Th228','Ra224','Pb212','Bi212','Tl208']
col=['r','y','b','k','c']
for i in [1,2,3,4,5]:
    #print(non_filtered_th228_px.head())
    df_new=non_filtered_th228[non_filtered_th228.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;
    plt.hist(px_energy,200,density=False,histtype='step',alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Raw count ')
    plt.legend()
plt.title("Deposited Counts vs Energy for Th228 at all positions (No B.R. consideration)")
plt.show()

# Deposited Events plot without with B.R. considerations

In [ ]:
iso=['Th228','Ra224','Pb212','Bi212','Tl208']
col=['r','y','b','k','c']
for i in [1,2,3,4,5]:
    #print(non_filtered_th228_px.head())
    df_new=non_filtered_th228[non_filtered_th228.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;
    plt.hist(px_energy,200,density=False,histtype='step',weights=df_new.weight,alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('G4tree counts ')
    plt.legend()
plt.title("Deposited Counts vs Energy for Th228 at all positions (B.R. consideration)")
plt.show()

# SS (GOOD EVENTS) without B.R.

In [ ]:
iso=['Th228','Ra224','Pb212','Bi212','Tl208']
col=['r','y','b','k','c']
for i in [1,2,3,4,5]:
    #print(non_filtered_th228_px.head())
    df_new=df_check_cut[df_check_cut.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;
    plt.hist(px_energy,20,density=False,histtype='step',alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('G4tree counts ')
    plt.legend()
plt.title("Good Counts vs Energy for Th228 at all positions (No B.R. consideration)")
plt.show()

# SS (GOOD EVENTS) with B.R.

In [ ]:
iso=['Th228','Ra224','Pb212','Bi212','Tl208']
col=['r','y','b','k','c']
for i in [1,2,3,4,5]:
    #print(non_filtered_th228_px.head())
    df_new=df_check_cut[df_check_cut.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;
    plt.hist(px_energy,5,density=False,histtype='step',weights=df_new.weight,alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Raw count ')
    plt.legend()
plt.title("Good Counts vs Energy for Th228 at all positions (No B.R. consideration)")
plt.show()

# DEC 2020 CUT

In [ ]:
#filtered values
fil_th228=get_filtered(th228_all,dec102020_filter)
#fil_th228=get_filtered(th228_all,jason_filter)

#(abs(fil_th228['energy']-2614)<10).sum()

In [ ]:
fil_th228.isotope.value_counts()

In [ ]:
good_events=fil_th228.shape[0]
print(f"good events: {good_events}")
gamma=good_events/total_simulations
print(f"gamma: {gamma:e}")

In [ ]:
px_energy,px_weight=non_filtered_th228.energy,non_filtered_th228.weight
# nx_energy,nx_weight=non_filtered_th228_nx.energy,non_filtered_th228_nx.weight
# py_energy,py_weight=non_filtered_th228_py.energy,non_filtered_th228_py.weight
# ny_energy,ny_weight=non_filtered_th228_ny.energy,non_filtered_th228_ny.weight
# pz_energy,pz_weight=non_filtered_th228_pz.energy,non_filtered_th228_pz.weight
# nz_energy,nz_weight=non_filtered_th228_nz.energy,non_filtered_th228_nz.weight

In [ ]:
non_filtered_th228.head()

In [ ]:
px_energy.min(),px_energy.max(),len(px_energy)

In [ ]:
plt.hist(px_energy,200,density=False,histtype='step',facecolor='g',alpha=0.75)
plt.xlabel('Energy [keV]')
plt.ylabel('Raw count ')
plt.show()

In [ ]:
#with dec2020 cut
iso=['th228','ra224','pb212','bi212','tl208']
col=['r','y','y','k','c']
filtered_th228=get_filtered(th228_all,dec102020_filter)
print(f"Total rows: {filtered_th228.shape[0]}")
for i in [1,2,3,4,5]:
    #print(non_filtered_th228_px.head())
    df_new=filtered_th228[filtered_th228.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;weight=df_new.weight
    plt.hist(px_energy,2037,density=False,weights=weight,histtype='step',alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Raw count ')
    plt.legend()
plt.title("SS Energy for Th228 at all positions")
plt.show()

In [ ]:
#with the filter used by Jason
iso=['th228','ra224','pb212','bi212','tl208']
col=['r','y','y','k','c']
filtered_th228=get_filtered(th228_all,jason_filter)
print(f"Total rows: {filtered_th228.shape[0]}")
for i in [1,2,3,4,5]:
    #print(non_filtered_th228_px.head())
    df_new=filtered_th228[filtered_th228.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;weight=df_new.weight
    plt.hist(px_energy,534,density=True,weights=weight,histtype='step',alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Raw count ')
    plt.legend()
plt.title("SS Energy for Th228 at all positions")
plt.show()

In [ ]:
####### ################################## STOP HERE FOR NOW

In [ ]:
filtered_th228_px=get_filtered(th228_px,dec102020_filter)
filtered_th228_px.head()
# filtered_th228_nx=get_filtered(th228_nx,dec102020_filter)
# filtered_th228_py=get_filtered(th228_py,dec102020_filter)
# filtered_th228_ny=get_filtered(th228_ny,dec102020_filter)
# filtered_th228_pz=get_filtered(th228_pz,dec102020_filter)
# filtered_th228_nz=get_filtered(th228_nz,dec102020_filter)

In [ ]:
# #total=10**6
# #fig, ax = plt.subplots(1, 1,figsize=(10,6))
# plt.plot(px_energy)
# #my_plotter(ax,px_energy,px_weight*total,"PX","log")
# # my_plotter(ax,nx_energy,nx_weight,"NX","log")
# # my_plotter(ax,py_energy,py_weight,"PY","log")
# # my_plotter(ax,ny_energy,ny_weight,"NY","log")
# # #plt.xlim(0,5000)
# plt.grid()
# #plt.savefig("rawlogxy.pdf")
# plt.show()

# WITH FILTER

In [ ]:
plot_isotopes(non_filtered_th228_px)

In [ ]:
filtered_th228_px=get_filtered(th228_px,dec102020_filter)
# filtered_th228_nx=get_filtered(th228_nx,dec102020_filter)
# filtered_th228_py=get_filtered(th228_py,dec102020_filter)
# filtered_th228_ny=get_filtered(th228_ny,dec102020_filter)
# filtered_th228_pz=get_filtered(th228_pz,dec102020_filter)
# filtered_th228_nz=get_filtered(th228_nz,dec102020_filter)

In [ ]:
filtered_th228_px.head()

In [ ]:
choose_list=[non_filtered_th228_px]#,non_filtered_th228_nx,non_filtered_th228_py,non_filtered_th228_ny,non_filtered_th228_pz,non_filtered_th228_nz]
for i in choose_list:
    print(f"{get_rows(i)}")

In [ ]:
for i in [filtered_th228_px,filtered_th228_nx,filtered_th228_py,filtered_th228_ny,filtered_th228_pz,filtered_th228_nz]:
    print(f"{get_rows(i)}")

In [ ]:
# for i in ['p','n']:
#     for j in ['x','y']:
#         df_name=f'filtered_th228_{i}{j}'
#         print(f"df name :{df_name}")
        
# #filtered_th228_py.shape

In [ ]:
px_energy,px_weight=filtered_th228_px.energy,filtered_th228_px.weight
# nx_energy,nx_weight=filtered_th228_nx.energy,filtered_th228_nx.weight
# py_energy,py_weight=filtered_th228_py.energy,filtered_th228_py.weight
# ny_energy,ny_weight=filtered_th228_ny.energy,filtered_th228_ny.weight
# pz_energy,pz_weight=filtered_th228_pz.energy,filtered_th228_pz.weight
# nz_energy,nz_weight=filtered_th228_nz.energy,filtered_th228_nz.weight

In [ ]:
#filtered_th228_pz.columns

In [ ]:

fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,px_energy,px_weight,"PX","log")
# my_plotter(ax,nx_energy,nx_weight,"NX","log")
# my_plotter(ax,py_energy,py_weight,"PY","log")
# my_plotter(ax,ny_energy,ny_weight,"NY","log")
plt.grid()
#plt.savefig("logxy.pdf")
plt.show()

In [ ]:

# fig, ax = plt.subplots(1, 1,figsize=(10,6))
# my_plotter(ax,pz_energy,pz_weight,"PZ","log")
# my_plotter(ax,nz_energy,nz_weight,"NZ","log")
# plt.grid()
# #plt.savefig("logz.pdf")
# plt.show()

In [ ]:

fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,px_energy,px_weight,"PX","log")
my_plotter(ax,nx_energy,nx_weight,"NX","log")
my_plotter(ax,py_energy,py_weight,"PY","log")
my_plotter(ax,ny_energy,ny_weight,"NY","log")
my_plotter(ax,pz_energy,pz_weight,"PZ","log")
my_plotter(ax,nz_energy,nz_weight,"NZ","log")
plt.grid()
plt.savefig("logxyz.pdf")
plt.show()

In [ ]:

fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,px_energy,px_weight,"PX","linear")
my_plotter(ax,nx_energy,nx_weight,"NX","linear")
my_plotter(ax,py_energy,py_weight,"PY","linear")
my_plotter(ax,ny_energy,ny_weight,"NY","linear")
plt.grid()
plt.savefig("linearxy.pdf")
plt.show()

In [ ]:

fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,pz_energy,pz_weight,"PZ","linear")
my_plotter(ax,nz_energy,nz_weight,"NZ","linear")
plt.grid()
plt.savefig("linearz.pdf")
plt.show()

In [ ]:
scale_kind='linear'
fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,px_energy,px_weight,"PX",scale_kind)
my_plotter(ax,nx_energy,nx_weight,"NX",scale_kind)
my_plotter(ax,py_energy,py_weight,"PY",scale_kind)
my_plotter(ax,ny_energy,ny_weight,"NY",scale_kind)
my_plotter(ax,pz_energy,pz_weight,"PZ",scale_kind)
my_plotter(ax,nz_energy,nz_weight,"NZ",scale_kind)
plt.grid()
plt.savefig("linearxyz.pdf")
plt.show()

# ISOTOPES

In [ ]:
#unique isotopes
filtered_th228_px.isotope.nunique()

In [ ]:
#value counts
filtered_th228_px.isotope.value_counts()

In [ ]:
#dataframe for isotopes
filtered_th228_px_bi212=filtered_th228_px.query('isotope==1')
filtered_th228_px_tl208=filtered_th228_px.query('isotope==2')

In [ ]:
px_energy_bi212,px_weight_bi212=filtered_th228_px_bi212.energy,filtered_th228_px_bi212.weight
px_energy_tl208,px_weight_tl208=filtered_th228_px_tl208.energy,filtered_th228_px_tl208.weight
# nx_energy,nx_weight=filtered_th228_nx.energy,filtered_th228_nx.weight
# py_energy,py_weight=filtered_th228_py.energy,filtered_th228_py.weight
# ny_energy,ny_weight=filtered_th228_ny.energy,filtered_th228_ny.weight

In [ ]:

fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,px_energy_bi212,px_weight_bi212,"PX-Bi212","log")
my_plotter(ax,px_energy_tl208,px_weight_tl208,"PX-Tl208","log")
# my_plotter(ax,nx_energy,nx_weight,"NX","log")
# my_plotter(ax,py_energy,py_weight,"PY","log")
# my_plotter(ax,ny_energy,ny_weight,"NY","log")
plt.grid()
#plt.savefig("log.pdf")
plt.show()

In [ ]:
sc='linear'
fig, ax = plt.subplots(1, 1,figsize=(6,4))
my_plotter(ax,px_energy_bi212,px_weight_bi212,"PX-Bi212",sc)
my_plotter(ax,px_energy_tl208,px_weight_tl208,"PX-Tl208",sc)
# my_plotter(ax,nx_energy,nx_weight,"NX","log")
# my_plotter(ax,py_energy,py_weight,"PY","log")
# my_plotter(ax,ny_energy,ny_weight,"NY","log")
plt.grid()
#plt.savefig("log.pdf")
plt.show()

In [ ]:
plot_isotopes(filtered_th228_px,'PX')
plot_isotopes(filtered_th228_nx,'NX')
plot_isotopes(filtered_th228_py,'PY')
plot_isotopes(filtered_th228_ny,'NY')

In [ ]:
# file='Example_Bi212_pos648.5_0_-1022.6_seed1.nEXOevents.root'
# suffix=":Event/Sim/SimEvent/SimEvent"
root='/home/thakur/slac_data/PX1M/'

In [ ]:
file='PX1M_Th228_pos648.5_0_-1022.6.root'+':tree'

In [ ]:
file1=root+file
file1

In [ ]:
f=uproot.open(file1)

In [ ]:
f.values()

In [ ]:
f.keys()

In [ ]:
f.show()

In [ ]:
df=f.arrays(f.keys(),library='pd')

In [ ]:
df

In [ ]:
df.columns.values

In [ ]:
apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'
#app='(m_nOPCal< (1.064*m_nQ+703))'

In [ ]:
df_filtered=df.query(apply_filter).reset_index(drop=True)

In [ ]:
df_filtered

In [ ]:
ene, weight = df_filtered.energy,df_filtered.weight
component='energy'
isotope='Th228'
location='PX'
plt.hist(ene, bins=200, histtype=u'step', weights=weight, density=True, label=component)
plt.yscale("log")
plt.ylabel("Counts")
plt.xlabel("Energy [keV]")
plt.title(f"{isotope} [{location}] Energy PDF for SS events in the inner 2000 kg")
plt.legend()
plt.show()

In [ ]:
ene, weight = df_filtered.energy,df_filtered.weight
component='energy'
isotope='Th228'
location='PX'
plt.hist(ene, bins=200, histtype=u'step', weights=weight, density=True, label=component)
plt.yscale("log")
plt.ylabel("Counts")
plt.xlabel("Energy [keV]")
plt.title(f"{isotope} [{location}] Energy PDF for SS events in the inner 2000 kg")
plt.legend()
plt.show()

In [ ]:
ene, weight = df_filtered.energy,df_filtered.weight
fig, ax = plt.subplots(1, 1,figsize=(10,6))
my_plotter(ax,ene,weight,"PX")
plt.show()

In [ ]:
# fil='sel_filter = (passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0)& (m_nOPCal < 1.064 * m_nQ + 703) & (m_nOPC
# al > 0.644 * m_nQ - 2411) & (~NESTBugFound)
# & (m_DNNvalue>0.85) & (standoff > 100))'

In [ ]:
en=df.energy.values
en

In [ ]:
# plt.hist(en,density=1,bins=500)
# plt.show()

In [ ]:
df.query('3000>energy>2000')['energy']

In [ ]:
energy=f.arrays(["energy"],library='np')

In [ ]:
energy

In [ ]:
fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(projection='3d')
ax.scatter(df['fGenX'],df['fGenY'],df['fGenZ'],color='r')
ax.set_xlabel('fGenX');ax.set_ylabel('fGenY');ax.set_zlabel('fGenZ')
plt.show()

In [ ]:
df['fGenParticleID'].value_counts()#.dropna().value_counts()

In [ ]:
df['fGenParticleID'].to_list()

In [ ]:
df['fGenParticleID'].unique

In [ ]:
from collections import Counter
from  itertools import chain
df_genparticle=pd.Series(Counter(chain(df.fGenParticleID)))
dfgen=df_genparticle.to_frame()
#dfgen=dfgen.rename(columns=['count'])
#df_genparticle.rename(columns=['iso','count'])
dfgen.columns=['num'];#dfgen.index='iso'

In [ ]:
dfgen.plot(kind='bar')

In [ ]:
selection='fXpos'
x=df[selection]
for y in x:
    print(f"length of y[0] {len(y[0])}")
    print(f"y[0] values {y[0]}")


In [ ]:
selection='fYpos'
y=df[selection]
y

In [ ]:
selection='fZpos'
z=df[selection]
z